In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import tensorflow as tf
from sciann import Functional, Variable, SciModel, PDE
from sciann.utils import *
import scipy.io 
import time
import random
from scipy import interpolate
from scipy import ndimage
from scipy.interpolate import griddata

from eikonalfm import factored_fast_marching as ffm
from eikonalfm import distance
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

2024-07-10 19:58:53.555370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 19:58:53.704416: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-10 19:58:53.735952: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 19:58:54.252524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

---------------------- SCIANN 0.7.0.0 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.10.1 
Python Version: 3.8.16 (default, Jun 12 2023, 18:09:05) 
[GCC 11.2.0] 

Outdated SciANN installation is found (V-0.7.0.0). Get the latest version (V-0.7.0.1):  
      > pip [--user] install --upgrade sciann  


In [2]:
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
np.random.seed(123)
tf.random.set_seed(123)

In [4]:
ymin = 0.; ymax = 3; deltay = 0.025;
xmin = 0.; xmax = 4; deltax = 0.025;
zmin = 0.; zmax = 2.05; deltaz = 0.025;

mat_data = scipy.io.loadmat('./3Dvelocity/velocity_3d.mat')
vp = mat_data['v']
vs = vp/1.731

nz = vp.shape[0]
nx = vp.shape[1]
ny = vp.shape[2]


In [5]:
vp_true = vp
vs_true = vs
print(vp_true.max(),vp_true.min())
print(vs_true.max(),vs_true.min())
print("taup1 range:",vp_true.min()/vp_true.max(),vp_true.max()/vp_true.min())
print("taus1 range:",vs_true.min()/vs_true.max(),vs_true.max()/vs_true.min())

5.9962373 2.447329
3.464031 1.4138238
taup1 range: 0.40814412 2.450115
taus1 range: 0.40814412 2.450115


In [6]:
#jianboqi
iz = 0.
# ix = np.round(np.arange(0,4.01,0.1),2); 
# iy = np.round(np.arange(0,3.01,0.1),2);
ix = np.round(np.arange(0,4.01,0.2),2); 
iy = np.round(np.arange(0,3.01,0.2),2);
sdata = np.load('random_coordinates.npz')
sx = sdata['rx']
sy = sdata['ry']
sz = sdata['rz']

In [7]:
ssx,ssy = np.meshgrid(ix,iy,indexing='ij')
ssx = ssx.reshape(-1,1)
ssy = ssy.reshape(-1,1)

In [8]:
iX,iY,rssz= np.meshgrid(ix,iy,sz,indexing='ij')
iX,iY,rssx= np.meshgrid(ix,iy,sx,indexing='ij')
iX,iY,rssy= np.meshgrid(ix,iy,sy,indexing='ij')
ss = [iX.reshape(-1,1),iY.reshape(-1,1)]
rssz = rssz.reshape(-1,1)
rssx = rssx.reshape(-1,1)
rssy = rssy.reshape(-1,1)

In [9]:
num = 500000
rx = np.round(np.random.uniform(xmin, xmax, num), 4).reshape(-1,1)
ry = np.round(np.random.uniform(ymin, ymax, num), 4).reshape(-1,1)
rz = np.round(np.random.uniform(zmin, zmax, num), 4).reshape(-1,1)
rsx = np.round(np.random.uniform(xmin, xmax, num), 4).reshape(-1,1)
rsy = np.round(np.random.uniform(ymin, ymax, num), 4).reshape(-1,1)
rsz = np.round(np.random.uniform(zmin, zmax, num), 4).reshape(-1,1)

In [10]:
rx = np.append(rx,ss[0])  
ry = np.append(ry,ss[1])
rz = np.append(rz,np.zeros(ss[0].shape))
rsz = np.append(rsz, rssz)
rsx = np.append(rsx, rssx)
rsy = np.append(rsy, rssy)

In [11]:
step = 0.025
rz_new = (rsz / step).astype(int)
rx_new = (rsx / step).astype(int)
ry_new = (rsy / step).astype(int)

rvelp_list = []
rvels_list = []

for rz_, rx_ ,ry_ ,in zip(rz_new, rx_new, ry_new):
    rvelp_list.append(vp[rz_,rx_,ry_])
    rvels_list.append(vs[rz_,rx_,ry_])
rvelp = np.array(rvelp_list)
rvels = np.array(rvels_list)

In [12]:
# rvelq = np.ones(rx.shape)*3;
T0p = np.sqrt((rz-rsz)**2 + (rx-rsx)**2 + (ry-rsy)**2)/rvelp; 
px0p = np.divide(rx-rsx, T0p*rvelp**2, out=np.zeros_like(T0p),where=T0p!=0)
pz0p = np.divide(rz-rsz, T0p*rvelp**2, out=np.zeros_like(T0p),where=T0p!=0)
py0p = np.divide(ry-rsy, T0p*rvelp**2, out=np.zeros_like(T0p),where=T0p!=0)

T0s = np.sqrt((rz-rsz)**2 + (rx-rsx)**2 + (ry-rsy)**2)/rvels; 
px0s = np.divide(rx-rsx, T0s*rvels**2, out=np.zeros_like(T0s),where=T0s!=0)
pz0s = np.divide(rz-rsz, T0s*rvels**2, out=np.zeros_like(T0s),where=T0s!=0)
py0s = np.divide(ry-rsy, T0s*rvels**2, out=np.zeros_like(T0s),where=T0s!=0)

In [13]:
indices = np.arange(len(rx) -ss[0].size  , len(rx))
index = np.where(np.isin(np.arange(len(rx)), indices))[0]

print(index.shape)

(16800,)


In [14]:
Tp_data=np.load('Tp_suiji.npy')
Ts_data=np.load('Ts_suiji.npy')

In [15]:
x_n = (ssx / deltax).astype(int)
y_n = (ssy / deltay).astype(int)

Tp_list = []
Ts_list = []

for rx_, ry_ in zip(x_n, y_n):
    Tp_list.append(Tp_data[0,rx_,ry_,:])
    Ts_list.append(Ts_data[0,rx_,ry_,:])
    
Tp = np.array(Tp_list)
Tp = Tp.reshape(-1,1)

Ts = np.array(Ts_list)
Ts = Ts.reshape(-1,1)

In [16]:

K.clear_session()

hidden_layers = 8*[64]
hidden_layers2 = 8*[32]

xt = Variable("xt", dtype='float64')
zt = Variable("zt", dtype='float64')
yt = Variable("yt", dtype='float64')
sxt = Variable("sxt", dtype='float64')
syt = Variable("syt", dtype='float64')
szt = Variable("szt", dtype='float64')
px0pt = Variable("px0pt", dtype='float64')
pz0pt = Variable("pz0pt", dtype='float64')
py0pt = Variable("py0pt", dtype='float64')
T0pt = Variable("T0pt", dtype='float64')
px0st = Variable("px0st", dtype='float64')
pz0st = Variable("pz0st", dtype='float64')
py0st = Variable("py0st", dtype='float64')
T0st = Variable("T0st", dtype='float64')

taup, taus = Functional(['taup','taus'], [zt, xt, yt, szt, sxt, syt], hidden_layers, 'l-elu','sigmoid')
vp, vs = Functional(['vp','vs'], [zt, xt, yt], hidden_layers2, 'l-elu','sigmoid')

taup = taup*3.0
taus = taus*3.0
vp = vp*6.0
vs = vs*4.0

Lp = (T0pt * diff(taup, xt) + taup * px0pt) ** 2 + (T0pt * diff(taup, zt) + taup * pz0pt) ** 2 + (T0pt * diff(taup, yt) + taup * py0pt) ** 2 - 1.0 / vp ** 2
Ls = (T0st * diff(taus, xt) + taus * px0st) ** 2 + (T0st * diff(taus, zt) + taus * pz0st) ** 2 + (T0st * diff(taus, yt) + taus * py0st) ** 2 - 1.0 / vs ** 2

imposed_ids = index
imposed_taup = (Tp[index-num]).reshape(-1,1)
imposed_taus = (Ts[index-num]).reshape(-1,1)

targets = [taup*T0pt, taus*T0st, Lp, Ls]
target_vals = [(imposed_ids, imposed_taup),(imposed_ids, imposed_taus),'zeros','zeros']

model = SciModel(
    [zt, xt, yt, szt, sxt, syt, pz0pt, px0pt, py0pt, T0pt, pz0st, px0st, py0st, T0st],
    targets
)

In [17]:
epoch_num = 200
start_time = time.time()
hist = model.train(
    [rz, rx, ry, rsz, rsx, rsy, pz0p, px0p, py0p, T0p, pz0s, px0s, py0s, T0s],
    target_vals,
    batch_size=5000,
    epochs=epoch_num,
    learning_rate=0.005,
    verbose=2,
)

elapsed = time.time() - start_time
print('Training time: %.2f minutes' % (elapsed / 60.))


Total samples: 516800 
Batch size: 5000 
Total batches: 104 



2024-07-10 19:58:59.471945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 19:59:00.041797: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45590 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:73:00.0, compute capability: 8.6
2024-07-10 19:59:00.074060: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Epoch 1/2000
104/104 - 7s - loss: 0.0905 - mul_16_loss: 0.0135 - mul_17_loss: 0.0345 - sub_loss: 0.0021 - sub_1_loss: 0.0400 - lr: 0.0050 - time: 7.0965 - 7s/epoch - 68ms/step
Epoch 2/2000
104/104 - 3s - loss: 0.0064 - mul_16_loss: 0.0012 - mul_17_loss: 0.0037 - sub_loss: 2.5448e-04 - sub_1_loss: 0.0013 - lr: 0.0050 - time: 3.3374 - 3s/epoch - 32ms/step
Epoch 3/2000
104/104 - 3s - loss: 0.0054 - mul_16_loss: 0.0010 - mul_17_loss: 0.0031 - sub_loss: 2.0158e-04 - sub_1_loss: 0.0010 - lr: 0.0050 - time: 3.4947 - 3s/epoch - 34ms/step
Epoch 4/2000
104/104 - 3s - loss: 0.0047 - mul_16_loss: 8.8579e-04 - mul_17_loss: 0.0028 - sub_loss: 1.6620e-04 - sub_1_loss: 9.0692e-04 - lr: 0.0050 - time: 3.4979 - 3s/epoch - 34ms/step
Epoch 5/2000
104/104 - 3s - loss: 0.0039 - mul_16_loss: 7.2829e-04 - mul_17_loss: 0.0023 - sub_loss: 1.3399e-04 - sub_1_loss: 8.2330e-04 - lr: 0.0050 - time: 3.4930 - 3s/epoch - 34ms/step
Epoch 6/2000
104/104 - 3s - loss: 0.0033 - mul_16_loss: 5.8192e-04 - mul_17_loss: 0.0018

In [18]:
model.save_weights('pinntomo_over3d_ori.h5')

In [19]:
loss_tomo_ori = hist.history['loss']
np.save('./loss_tomo_over_random_ori_sparse%d.npy'%(epoch_num),loss_tomo_ori)

In [20]:
loss_tomo_new = np.load('./loss_tomo_over_random_new%d.npy'%(epoch_num))

plt.style.use('default')

plt.figure(figsize=(8,5))

ax = plt.gca()
plt.semilogy(loss_tomo_new[::1],'r')
plt.semilogy(loss_tomo_ori[::1],'k')

plt.xlabel('epochs', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.legend(['New','Original'], fontsize=12)
plt.grid()
# plt.savefig("./Fig/over3d_loss_sparse.jpeg", format='jpeg', bbox_inches="tight",dpi=400)

FileNotFoundError: [Errno 2] No such file or directory: './loss_tomo_over_random_new2000.npy'